# Scheinkorrelation bei abhängigen Messungen

Das Konzept der Korrelation bezeichnet, dass zwischen zwei Attributen eine Kopplung besteht.
Messungen sind bspw. dann voneinander abhängig, wenn man über einen Zeitverlauf ein Individuum beobachtet.
Dann kann man nämlich häufig davon ausgehen, dass Ereignisse aus der Vergangneheit die zukünftigen Messwerte beeinflussen werden.
Eine positive Korrelation hier, dass zwei Dinge gemeinsam auftreten, z. B. bedeutet die Zunahme von Autoverkehr in den Innenstädten eine gleichzeitige höhere Schadstoffbelastung.
Eine negative Korrelation bedeutet, dass die Kopplung umgekehrt ist.
Je höher der Bildungsstand von Frauen in einem Land ist, desto geringer fällt die Geburtenrate aus.
Eine Scheinkorrelation (Englisch: *spurious correlation*) bezeichnet den Fall, dass zwei Attribute rein zufällig miteinander korrelieren.
Hierfür gibt es auf http://www.tylervigen.com/spurious-correlations viele Beispiele.

In diesem Notebook wird gezeigt, weswegen Scheinkorrelationen für das Maschinelle Lernen eine Herausforderung darstellen.
Dabei schauen wir uns nun abhängige Messungen an.

In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor

Nun erstellen wir uns einen zufälligen Datensatz.
Er soll aber doch nicht ganz zufällig sein, damit wir das Seminar rechtzeitig abschließen können.
Deswegen setzen wir den Zufallsgenerator fest.
Dadurch hat jeder Durchlauf die gleichen Ergebnisse.

In [ ]:
random.seed(0)

## Exkurs: Der Random Walk

Ein Random Walk bezeichnet eine Reihe von Zufallszahlen, wobei der letzte vorliegende Wert als Ausgang genommen wird.
In der folgenden Implementierung wird für jeden Zeitschritt eine Münze geworfen, die entscheidet, ob der nächste Wert dem jetzigen Wert plus eins oder minus eins entspricht.

In [ ]:
def create_random_walk(n):
    "n: Länge des Random Walks"
    random_walk = []
    
    # Füge zufälliges Start-Element (-1 oder 1) ein
    random_walk.append(-1 if random.random() < 0.5 else 1)

    for i in range(n - 1):
        movement = -1 if random.random() < 0.5 else 1  # nach oben oder unten?
        value = random_walk[-1] + movement             # letzter Wert + neue Richtung
        random_walk.append(value)
    return random_walk

create_random_walk(5)

Dies ist sehr zufällig.
Zur Verdeutlichung werden nun 5 verschiedene Random Walks visualisiert.

In [ ]:
for i in range(5):
    plt.plot(create_random_walk(500))
    plt.show()

## Erstellung eines Datensatzes bestehend aus Random Walks

Nun erstellen wir uns einen Datensatz mit 26 verschiedenen Attributen.
Dies könnten zum Beispiel Sensormesswerte oder Eigenschaften von Benutzern sein.
Der Datensatz hat insgesamt 100 Reihen, sprich so viele verschiedene Messungen bzw. Anzahl von Benutzern.
Die Besonderheit hier ist nun, dass wir statt Zufallsvariablen Random Walks verwenden.
Dies entspricht eher der Annahme, dass wir in fixen Zeitintervallen Messungen vornehmen.
Hier entsprechen die Zeilen den Zeitschritten und die Spalten den verschiedenenen Messungen.
Das könnten entweder verschiedene Attribute eines Untersuchungsgegenstandes sein oder die verschiedenen Untersuchungsgegenstände selber.

In [ ]:
df = pd.DataFrame(data={
    column_name : create_random_walk(100)
    for column_name in list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
})
df

Auch hier hilft es, die Daten zuerst zu visualisieren.

In [ ]:
df.plot()
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()

Hier können wir auch die Korrelation zwischen den verschiedenen Attributen betrachten.

In [ ]:
fig = plt.figure(figsize=(19, 15))
plt.matshow(df.corr(), fignum=fig.number, cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(range(df.shape[1]), df.columns)
plt.yticks(range(df.shape[1]), df.columns)
plt.colorbar()
plt.show()

Es gibt klar ausgeprägte Scheinkorrelationen, sowohl positive als auch negative.
Eine Gruppe von Variablen (die hoch mit M korrelieren) ist hier einmal herausgepickt worden:

In [ ]:
df[list('FGHIMNQWX')].corr()

Hier kann man sehr gut sehen, wie F und M negativ korrelieren: 
Wenn F ansteigt, tendierd M dazu, abzufallen.
Dies kann man nun auch im Plot unten sehen:

In [ ]:
df[list('FMQWXZ')].plot()
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()
df[list('FM')].plot()
plt.legend(bbox_to_anchor=(1.1, 1.05))
plt.show()

## Auswirkungen auf das Maschinelle Lernen

Wie gehen die Lernalgorithmen hiermit um?
Die Ausgabe eines Regressors ist schließlich die Kombination der Eingabeattribute.

Hier wird $R^2$ (sprich: "R Squared") als Gütekriterium eingesetzt.
Weiterführende Infos gibt es z. B. bei 
[Wikipedia](https://en.wikipedia.org/wiki/Coefficient_of_determination).
Ein Wert von 1 bedeutet, dass das vorliegende Modell den Zielwert perfekt vorhersagt.
Negative Werte bedeuten, dass es besser gewesen wäre, wenn der Durchschnitt als Vorhersage gewählt worden wäre.
Dies bedeutet, dass das vorliegende Modell keinen Mehrwert bietet.

In [ ]:
regr = RandomForestRegressor(
    n_estimators=3,
    max_depth=2,
    random_state=1
)

X = df[list('FMQWXZ')].values
y = df["M"].values

train_scores = []
test_scores = []
for train_index, test_index in model_selection.TimeSeriesSplit(n_splits=10).split(X):
    regr.fit(X[train_index], y[train_index])
    train_score = regr.score(X[train_index], y[train_index])
    test_score = regr.score(X[test_index], y[test_index])
    train_scores.append(train_score)
    test_scores.append(test_score)

plt.plot(train_scores, label="Train")
plt.plot(test_scores, label="Test")
plt.show()
pd.DataFrame(data={
    "Training R2" : train_scores, 
    "Test R2" : test_scores
})

In Runde 2 und 3 beträgt der Wert von $R^2$ ca. 0,77.
Das ist je nach Anwendungsfall ein bereits ein gutes Ergebnis.
Alle Werte, die kleiner Null sind, zeigen, dass es besser gewesen wäre, statt dem Modell den Durchschnitt zu nehmen.
Dies wird hier noch einmal isoliert gezeigt.

In [ ]:
regr = RandomForestRegressor(
    n_estimators=3,
    max_depth=2,
    random_state=1
)
number_entries = len(df["M"])
regr.fit(X[0 : 30], 
         y[0 : 30]
)
regr.score(X[30 : 40],
           y[30 : 40])

Diesmal hat die Einteilung in Trainings- und Testset dabei geholfen, ein schlechtes Modell als solches zu identifizieren.
Wenn wir aber Pech gehabt hätten, hätten nur die Daten vorgelegen, die eine Korrelation aufgezeigt haben.
Deswegen benötigt Maschinelles Lernen Daten über einen langen Zeitraum.
Denn wenn die Zeitreihen lang genug sind, wird sich durch Zufall höchstwahrscheinlich die Scheinkorrelation von alleine auflösen.

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0; display:inline" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a> &nbsp;&nbsp;&nbsp;&nbsp;Dieses Werk von Marvin Kastner ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Namensnennung 4.0 International Lizenz</a>.